In [ ]:
# # save data locally as a json file
# import json

# with open("data.json", "w") as f:
#     json.dump(data_, f, indent=4)

NameError: name 'data_' is not defined

In [3]:
#load data from json file
with open("data.json", "r") as f:
    data = json.load(f)

In [4]:
from sentence_transformers import SentenceTransformer, util
import heapq

# Load Sentence Transformer model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

# Function to calculate harmonic mean
def harmonic_mean(a, b):
    return (2 * a * b) / (a + b) if a + b != 0 else 0

# Function to get embeddings
def get_embedding(text):
    return model.encode(text, convert_to_tensor=True)

# Retrieval pipeline
def retrieve(query, data, top_n=5):
    # Get query embedding
    query_embedding = get_embedding(query)
    
    # Step 1: Calculate similarity with topics
    topic_similarities = []
    for idx, item in enumerate(data):
        topic_embedding = get_embedding(item['topic'])
        similarity = util.cos_sim(query_embedding, topic_embedding).item()
        topic_similarities.append((similarity, idx))
    
    # Step 2: Select top N topics
    top_topics = heapq.nlargest(top_n, topic_similarities, key=lambda x: x[0])
    
    # Step 3: Calculate similarity with content for top topics
    results = []
    for topic_similarity, idx in top_topics:
        content_embedding = get_embedding(data[idx]['content'])
        content_similarity = util.cos_sim(query_embedding, content_embedding).item()
        # Step 4: Rank by harmonic mean
        hmean = harmonic_mean(topic_similarity, content_similarity)
        results.append((hmean, data[idx]))
    
    # Step 5: Sort results by harmonic mean
    results = sorted(results, key=lambda x: x[0], reverse=True)
    
    return results

/Users/azim/Desktop/CourseMate/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# User Query
query = "what is the energy in fully developed laminar flow?"

# Retrieve Results
results = retrieve(query, data)

# Print Ranked Results
print("Ranked Results:")
for score, item in results:
    print(f"Total Similarity: {score:.4f}, Topic: {item['topic']}, Content: {item['content'][:1000]}...")

Ranked Results:
Total Similarity: 0.6317, Topic: Energy in Fully Developed Laminar Flow, Content: 
**Key Concepts**:
1. **Bernoulli's Equation for Energy Loss**:
   - For uniform velocity profiles (\( lpha = 1 \)):
     $$ rac{p_1}{\gamma} + z_1 - rac{p_2}{\gamma} + z_2 = h_L $$
   - Head loss (\( h_L \)) accounts for energy dissipation due to viscous effects.

2. **Head Loss in Laminar Flow**:
   - Using pressure drop and wall shear stress:
     $$ h_L = rac{4 l 	au_w}{\gamma D} $$
   - This formula is valid for both laminar and turbulent flow.

**Key Equation**:
- Head loss in terms of wall shear stress:
  $$ h_L = rac{4 l 	au_w}{\gamma D} $$
...
Total Similarity: 0.4931, Topic: Introduction to Laminar and Turbulent Flows, Content: 
The lecture introduces laminar and turbulent flows with examples and key characteristics.

1. **Observation Example**:
   - A candle's smoke plume initially rises smoothly (laminar flow) and then fluctuates randomly (turbulent flow).

2. **Definition